In [2]:
#!pip install git+https://github.com/huggingface/transformers sentencepiece datasets librosa speechbrain

from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
inputs = processor(text="History’s primary purpose is to stand at the centre of diverse, tolerant, intellectually rigorous debate about our existence: our political systems, leadership, society, economy and culture.", return_tensors="pt")

# load xvector containing speaker's voice characteristics from a dataset

speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

sf.write("speech.wav", speech.numpy(), samplerate=16000)

In [6]:
import IPython
IPython.display.Audio("speech.wav")

Generate speech embedding

In [7]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name, 
    run_opts={"device": device}, 
    savedir=os.path.join("/tmp", spk_model_name)
)

def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

In [8]:
"""
from datasets import load_dataset, Audio, Dataset

dataset = load_dataset(
    "facebook/voxpopuli", "nl", data_files="data/en/train/train_part_0.tar.gz"
)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
"""

In [9]:
audio_dataset = Dataset.from_dict({"audio": ["/root/audio/nora.wav"]}).cast_column("audio", Audio(sampling_rate=16000))
audio = audio_dataset[0]["audio"]
my_speaker_embeddings = create_speaker_embedding(audio["array"])

In [15]:
inputs = processor(text="Perhaps we might even distill patterns and lessons to guide", return_tensors="pt")

# load xvector containing speaker's voice characteristics from a dataset

#speaker_embeddings = torch.tensor(embeddings_dataset[500]["xvector"]).unsqueeze(0)
speaker_embeddings = torch.tensor(my_speaker_embeddings).unsqueeze(0)

speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

sf.write("speech.wav", speech.numpy(), samplerate=16000)

In [16]:
import IPython
IPython.display.Audio("speech.wav")